# Induced gap and magnetic field

# TO-DO:
* Remove the use of `delta` function in `tunnel_hoppings`
* Correct for zero supercurrent because of non-zero total $A$ in SC, example from [orbitalfield](https://github.com/basnijholt/orbitalfield) project:

```python
    def peierls(val, ind, a):
        def phase(s1, s2, p):
            x, y, z = s1.pos
            A_site = [p.B_y * z - p.B_z * y, 0, p.B_x * y][ind]
            if p.A_correction:
                A_sc = [A(p, *site.pos) for site in sc_sites]
                A_site -= np.mean(A_sc, axis=0)[ind]
            A_site *= a * 1e-18 * eV / hbar
            return np.cos(A_site) * s0s0 - 1j * np.sin(A_site) * s0sz
```

In [ ]:
# 1. Standard library imports
from types import SimpleNamespace
from importlib import reload

# 2. External package imports
import holoviews as hv
import kwant
import matplotlib.pyplot as plt
import numpy as np
import sympy

# 3. Internal imports
import funcs
reload(funcs)

hv.notebook_extension('bokeh')

# Usage

In [ ]:
ham_pars = dict(alpha=20, B_x=0, B_y=0, B_z=0, Delta=0.250, g=50, mu_B=funcs.constants.mu_B,
                orbital=True, c=funcs.constants.c, hbar=funcs.constants.hbar,
                m_eff=funcs.constants.m_eff, mu_sc=0, mu_sm=0, e=funcs.constants.e, c_tunnel=1)

syst_params = dict(a=10, angle=45, onsite_disorder=False,
                   L=200, phi=135, r1=50, r2=80, shape='circle',
                   with_leads=True, with_shell=True)

syst = funcs.make_3d_wire(**syst_params)
kwant.plot(syst)

In [ ]:
%%opts Path [aspect='square']
lead = funcs.make_lead(10, 50, 70, 135, 0, True, 'circle')
ham_pars['B_x'] = 1
ham_pars['B_y'] = 1
ham_pars['orbital'] = True
ks = np.linspace(-3, 3)
Es = funcs.bands(lead, ham_pars, ks)
p1 = hv.Path((ks, Es))[:, -100:100]

ham_pars['orbital'] = False
Es1 = funcs.bands(lead, ham_pars, ks)
p2 = hv.Path((ks, Es1))[:, -100:100]

p1 * p2

In [ ]:
gap = funcs.find_gap(lead, ham_pars)
print('The bandgap is {} meV'.format(gap))

In [ ]:
ham_pars = dict(alpha=20, B_x=0, B_y=0, B_z=0, Delta=0., g=50, mu_B=funcs.constants.mu_B,
                orbital=True, c=funcs.constants.c, hbar=funcs.constants.hbar,
                m_eff=funcs.constants.m_e, mu_sc=0, mu_sm=0, e=funcs.constants.e, c_tunnel=1)

S = kwant.smatrix(syst, params=ham_pars)

In [ ]:
funcs.andreev_conductance(syst, ham_pars)